# Exploratory Data Analysis

Closer look on the biggest available datasets about New York's taxis rides.

In [ ]:
# Frameworks
import pandas as pd
import numpy as np

In [11]:
df = pd.read_parquet(r'C:\Users\wikto\OneDrive\Dokumenty\AA_projects\road-optimization\data\yellow_tripdata_2025-01.parquet')

# DataFrame shape
print("-" * 30)
print(f"Dataframe shape: {df.shape}")

# All features
print("-" * 30)
print(f"All features: {df.columns}")

# Number of NaN's
print("-" * 30)
print(f"Number of NaN's: {df.isna().sum()}")

------------------------------
Dataframe shape: (3475226, 20)
------------------------------
All features: Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'Airport_fee',
       'cbd_congestion_fee'],
      dtype='object')
------------------------------
Number of NaN's: VendorID                      0
tpep_pickup_datetime          0
tpep_dropoff_datetime         0
passenger_count          540149
trip_distance                 0
RatecodeID               540149
store_and_fwd_flag       540149
PULocationID                  0
DOLocationID                  0
payment_type                  0
fare_amount                   0
extra                         0
mta_tax                       0
tip_amount      

| Feature | What information does it provide |
| :--- | :--- |
| **VendorID** | Kod dostawcy (1=CMT, 2=VeriFone) |
| **tpep_pickup_datetime** | Czas włączenia taksometru |
| **tpep_dropoff_datetime** | Czas wyłączenia taksometru |
| **passenger_count** | Liczba pasażerów |
| **trip_distance** | Dystans podróży w milach |
| **RatecodeID** | Kod taryfy (1=Standard, 2=JFK, 3=Newark, 5=Negocjowana) |
| **store_and_fwd_flag** | Czy rekord był zapisany w pamięci (Y/N) |
| **PULocationID** | Strefa rozpoczęcia kursu |
| **DOLocationID** | Strefa zakończenia kursu |
| **payment_type** | Płatność (1=Karta, 2=Gotówka, 3=Brak, 4=Spór) |
| **fare_amount** | Opłata taryfowa |
| **extra** | Dodatki (godziny szczytu / nocne) |
| **mta_tax** | Podatek MTA (0.50$) |
| **tip_amount** | Napiwek (tylko płatności kartą) |
| **tolls_amount** | Opłaty drogowe (mosty, tunele) |
| **improvement_surcharge** | Opłata modernizacyjna (0.30$) |
| **total_amount** | Suma całkowita (bez napiwków gotówkowych) |
| **congestion_surcharge** | Opłata za korki |
| **Airport_fee** | Opłata lotniskowa ($1.25) |
| **cbd_congestion_fee** | Opłata za wjazd do centrum (Manhattan) |

Potential columns that are important for our optimatization route model:
tpep_pickup_datetime, tpep_dropoff_datetime, trip_distance, RateCodeID, PULocationID, DOLocationID,
fare_amount, extra, tolls_amount, congestion_surcharge, airport_fee, cbd_congestion_fee


fare_amount - > koszt za przejechanie mili + czas spędzony w ruchu 

### trip_distance column analysis

In [ ]:
# Checking what unit trip_distance has
df['trip_distance'].head()  # --> CHANGE: mils to km's

# Is there negative values in fare_amount -> ujemne fare_amount może znaczyć, że przejazd był fałszywy - 
# np. kierowca zapomniał wyłączyć licznika
print((df['fare_amount'] < 0).value_counts()) # --> CHECK or DELETE negative columns

# Zwrócenie negatywnych wierszy i ich indeksów 
df_negative_fare = {'negative_fare': []}
df_negative_fare = pd.Series(df_negative_fare)

df_negative_fare = (df['fare_amount'] < 0)
df_negative_fare = df_negative_fare.loc[df_negative_fare]

# Indexe's of negative fare
for i in range(10):
    print(df_negative_fare.index[i])

# Check indexe's in main df
df.iloc[22]



fare_amount
False    3331108
True      144118
Name: count, dtype: int64
17
22
104
149
202
212
364
400
492
640


VendorID                                   2
tpep_pickup_datetime     2025-01-01 00:55:54
tpep_dropoff_datetime    2025-01-01 01:00:38
passenger_count                          1.0
trip_distance                           0.69
RatecodeID                               1.0
store_and_fwd_flag                         N
PULocationID                             137
DOLocationID                             233
payment_type                               4
fare_amount                             -6.5
extra                                   -1.0
mta_tax                                 -0.5
tip_amount                               0.0
tolls_amount                             0.0
improvement_surcharge                   -1.0
total_amount                           -11.5
congestion_surcharge                    -2.5
Airport_fee                              0.0
cbd_congestion_fee                       0.0
negative_fare_df                         NaN
Name: 22, dtype: object

### tpep_pickup_datetime and tpep_dropoff_datetime (time difference) columns analysis

In [ ]:
# Mean of time difference between pick up and drop off
df['difference'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
mean_diff = df['difference'].mean()
print(f'Mean of time difference: {mean_diff}')

# Outliers of time difference and their indexes
std_diff = df['difference'].std()
print(f'Std of time difference: {std_diff}')

lower_limit = mean_diff - 3*std_diff
upper_limit = mean_diff + 3*std_diff

upper_outliers = df[df['difference'] > upper_limit]
lower_outliers = df[df['difference'] < lower_limit]

# DELETE outliers




Mean of time difference: 0 days 00:15:01.086937
Std of time difference: 0 days 00:38:42.814931


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee,negative_fare_df,difference
2780,2,2025-01-01 00:18:22,2025-01-02 00:00:00,1.0,1.12,1.0,N,50,48,2,...,0.5,0.00,0.00,1.0,22.00,2.5,0.0,0.00,NaN,0 days 23:41:38
3982,2,2025-01-01 00:27:08,2025-01-02 00:00:00,1.0,5.72,1.0,N,137,33,1,...,0.5,4.56,0.00,1.0,34.96,2.5,0.0,0.00,NaN,0 days 23:32:52
5999,2,2025-01-01 01:09:24,2025-01-02 01:06:27,1.0,3.32,1.0,N,238,229,1,...,0.5,0.00,0.00,1.0,26.20,2.5,0.0,0.00,NaN,0 days 23:57:03
8811,2,2025-01-01 01:48:26,2025-01-02 01:41:57,2.0,0.99,1.0,N,163,143,1,...,0.5,2.04,0.00,1.0,15.64,2.5,0.0,0.00,NaN,0 days 23:53:31
9215,1,2025-01-01 01:04:01,2025-01-01 03:33:22,2.0,133.30,5.0,N,68,265,1,...,0.0,0.00,29.88,1.0,380.88,0.0,0.0,0.00,NaN,0 days 02:29:21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3374773,2,2025-01-26 17:21:56,2025-01-27 03:35:19,NaN,7.04,NaN,None,71,188,0,...,0.5,0.00,0.00,1.0,21.97,NaN,NaN,0.00,NaN,0 days 10:13:23
3417048,2,2025-01-29 21:39:23,2025-01-31 16:48:43,NaN,7.52,NaN,None,231,193,0,...,0.5,0.00,0.00,1.0,35.77,NaN,NaN,0.75,NaN,1 days 19:09:20
3428248,2,2025-01-30 11:38:52,2025-01-30 15:03:30,NaN,15.34,NaN,None,145,197,0,...,0.5,0.00,0.00,1.0,37.98,NaN,NaN,0.00,NaN,0 days 03:24:38
3446818,2,2025-01-31 07:05:52,2025-01-31 09:58:02,NaN,5.11,NaN,None,157,138,0,...,0.5,0.00,0.00,1.0,10.80,NaN,NaN,0.00,NaN,0 days 02:52:10
